In [82]:
get_processed_data_script_file = os.path.join(os.path.pardir,'src','data','get_processed_data.py')

In [83]:
%%writefile $get_processed_data_script_file

import pandas as pd
import numpy as np
import os

def read_data():
    
    raw_data_path = os.path.join(os.path.pardir,'data','raw')
    train_file_path = os.path.join(raw_data_path,'train.csv')
    test_file_path = os.path.join(raw_data_path,'test.csv')
    
    train_df = pd.read_csv(train_file_path,index_col='PassengerId')
    test_df = pd.read_csv(test_file_path,index_col='PassengerId')
    test_df['Survived'] = -888
    df =pd.concat((train_df,test_df),axis=0)
    return df

Overwriting ../src/data/get_processed_data.py


In [84]:
df = read_data()

/home/force/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [85]:
def process_data(df):
    return (df
            .assign(Title=lambda x:x.Name.map(get_title))
           .pipe(fill_missing_values)
           .assign(Fare_Bin = lambda x: pd.qcut(x.Fare, 4, labels =['very_low','low','high','very_high']))
           .assign(AgeState = lambda x: np.where(x.Age >=18 , 'Adult','Child'))
           .assign(FamilySize = lambda x: x.Parch+x.SibSp+1)
           .assign(IsMother = lambda x: np.where(((x.Sex=='femail') & (x.Parch>0) & (x.Age>18) & (x.Title!='Miss')),1,0))
           .assign(Cabin = lambda x: np.where(x.Cabin == 'T',np.nan,x.Cabin))
           .assign(Deck = lambda x :x.Cabin.map(get_deck))
           .assign(IsMale = lambda x:np.where(x.Sex == 'male',1,0))
           .pipe(pd.get_dummies,columns=['Deck','Pclass','Title','Fare_Bin','Embarked','AgeState'])
           
           .drop(['Cabin','Name','Ticket','Parch','SibSp','Sex'],axis=1)
           
           .pipe(reorder_columns))

def write_data(df):
    processed_data_path = os.path.join(os.path.pardir,'data','processed')
    write_train_path = os.path.join(processed_data_path,'train.csv')
    write_test_path = os.path.join(processed_data_path,'test.csv')
    
    df[df.Survived!=-888].to_csv(write_train_path)
    
    columns = [column for column in df.columns if column!='Survived']
    
    df[df.Survived==-888][columns].to_csv(write_test_path)
    
if __name__=='__main__':
    df=read_data()
    df=process_data(df)
    write_data(df)
    

/home/force/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [86]:
def get_title(name):
    title_group = {
        'mr':'Mr',
        'ms':'Mrs',
        'mrs':'Mrs',
        'miss':'Miss',
        'master':'Master',
        'don':'Sir',
        'rev':'Sir',
        'dr':'Officer',
        'mme':'Mrs',
        'major':'Officer',
        'lady':'Lady',
        'sir':'Sir',
        'mlle':'Miss',
        'col':'Officer',
        'capt':'Officer',
        'the countess':'Lady',
        'jonkheer':'Sir',
        'dona':'Lady'
    }
    first_name_with_title = name.split(',')[1]
    title = first_name_with_title.split('.')[0]
    title = title.strip().lower()
    return title_group[title]

def fill_missing_values(df):
    
    df.Embarked.fillna('C',inplace=True)
    
    median_fare = df [(df.Pclass==3) & (df.Embarked=='S')]['Fare'].median()
    df.Fare.fillna(median_fare,inplace=True)
    
    title_age_median = df.groupby('Title').Age.transform('median')
    df.Age.fillna(title_age_median,inplace=True)
    return df

def get_deck(cabin):
    return np.where(pd.notnull(cabin),str(cabin)[0].upper(),'z')

def reorder_columns(df):
    columns = [column for column in df.columns if column!='Survived']
    columns = ['Survived']+columns
    df = df[columns]
    return df

    
    

In [87]:
# process_data(df)

In [88]:
!python $get_processed_data_script_file


In [89]:
# train_df = pd.read_csv(write_train_path)
# train_df.info()

# test_df = pd.read_csv(write_test_path)
# test_df.info()